# W02 · Perceptron & Multilayer Perceptrons

**Learning objectives**

- Review the limitations of single-layer perceptrons and motivate deeper architectures.
- Practice reasoning about when to add layers versus adding more neurons to existing layers.
- Compare common activation functions and recognize how they influence optimization dynamics.
- Build intuition for diagnosing underfitting/overfitting when designing multilayer perceptrons (MLPs).

## Recap: From Perceptrons to MLPs

In Week 01 we trained a single dense layer on Fashion-MNIST. The perceptron learned a linear decision boundary over flattened pixels.

> **Key limitation:** Without hidden layers the model cannot express interactions between pixels. Decision boundaries remain linear, so patterns like curves, textures, or hierarchical shapes are hard to capture.

To move beyond linear separability we introduce **hidden layers**. Each additional layer composes nonlinear transformations that can
model progressively more abstract features. However, deeper networks add more parameters and optimization challenges, so we need a principled way to decide when to deepen versus widen a network.

## Heuristics for Adding Layers or Neurons

When designing an MLP, consider the following heuristics:

- **Problem complexity**: If your features interact in hierarchical ways (e.g., strokes composing clothing items), adding layers can help. If interactions are mostly linear, widening layers might suffice.
- **Bias vs. variance**:
  - High bias / underfitting → add layers or neurons, or change activations to increase capacity.
  - High variance / overfitting → reduce layers/neurons, add regularization, or collect more data.
- **Representation bottlenecks**: Too few neurons in a critical layer can restrict information flow. Gradually increase width while monitoring validation metrics.
- **Training dynamics**: Vanishing gradients or slow convergence could mean the architecture or activation is poorly matched; try `relu` or `elu` for deeper stacks, or add normalization.
- **Parameter budget / inference cost**: Deeper and wider networks cost more to train/serve. Aim for the simplest model that meets your performance target.

## Setup: Imports & Data Pipeline

We will reuse the Fashion-MNIST dataset from Week 01. To keep experiments fast, we'll train on a 12k-image subset and hold out 2k examples for validation.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np

print(tf.__version__)

# Load Fashion-MNIST
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

# Normalize to [0, 1]
train_images = train_images.astype("float32") / 255.0
test_images = test_images.astype("float32") / 255.0

# Flatten inputs for dense networks
train_images = train_images.reshape(-1, 28 * 28)
test_images = test_images.reshape(-1, 28 * 28)

# Create train/validation split (12k train / 2k val)
val_split = 2000
train_subset = 12000

train_x = train_images[:train_subset]
train_y = train_labels[:train_subset]
val_x = train_images[train_subset:train_subset + val_split]
val_y = train_labels[train_subset:train_subset + val_split]

class_names = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]

print(f"Training subset: {train_x.shape}, Validation subset: {val_x.shape}")

## Comparing Architectures: Depth vs. Width

We'll train several `tf.keras.Sequential` models that vary in depth (number of hidden layers) and width (neurons per layer). Each model trains for a small number of epochs so we can compare validation performance quickly.

In [ ]:
def build_mlp(hidden_layers, activation="relu", dropout_rate=0.0):
    model = keras.Sequential(name="mlp")
    model.add(layers.Input(shape=(28 * 28,)))
    for units in hidden_layers:
        model.add(layers.Dense(units, activation=activation))
        if dropout_rate > 0:
            model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

architectures = [
    ("shallow_wide", [256], "1 hidden layer, wide"),
    ("two_layer_balanced", [256, 128], "2 hidden layers"),
    ("deep_narrow", [128, 128, 64, 64], "4 hidden layers, narrower"),
]

histories = {}
for model_key, hidden_layers, description in architectures:
    print(f"
Training model: {model_key} ({description})")
    model = build_mlp(hidden_layers)
    history = model.fit(
        train_x,
        train_y,
        validation_data=(val_x, val_y),
        epochs=5,
        batch_size=128,
        verbose=0
    )
    histories[model_key] = history.history
    val_acc = history.history["val_accuracy"][-1]
    print(f"Final validation accuracy: {val_acc:.3f}")

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
for model_key, history in histories.items():
    epochs = range(1, len(history["accuracy"]) + 1)
    axes[0].plot(epochs, history["accuracy"], marker="o", label=model_key)
    axes[1].plot(epochs, history["val_accuracy"], marker="o", label=model_key)

axes[0].set_title("Training accuracy")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Accuracy")

axes[1].set_title("Validation accuracy")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Accuracy")
axes[1].legend()
plt.tight_layout()
plt.show()

### Discussion

- How quickly does each architecture reach strong validation accuracy?
- Which model shows signs of underfitting (low training + validation accuracy)?
- Do any models hint at overfitting (training accuracy increases while validation plateaus or drops)?

Use these observations to reason about when additional layers help versus when simply widening a single layer is enough.

## Underfitting vs. Overfitting in Practice

Architecture decisions can go wrong in both directions. We'll contrast a tiny network that lacks capacity with a very deep, wide network trained for longer without regularization to show how training and validation metrics diverge.

In [ ]:

extreme_configs = [
    ("underfit_tiny", [32], "Tiny capacity (underfitting)"),
    ("overfit_huge", [512, 512, 256, 256, 128, 128], "Deep and wide (overfitting)"),
]

extreme_histories = {}
for key, hidden_layers, description in extreme_configs:
    print(f"Training extreme model: {description}")
    model = build_mlp(hidden_layers)
    history = model.fit(
        train_x,
        train_y,
        validation_data=(val_x, val_y),
        epochs=20,
        batch_size=128,
        verbose=0
    )
    extreme_histories[key] = history.history
    final_train = history.history['accuracy'][-1]
    final_val = history.history['val_accuracy'][-1]
    print(f"Final train acc: {final_train:.3f}, val acc: {final_val:.3f}")


In [ ]:

def plot_history_curves(histories, title_prefix=""):
    metrics = [("accuracy", "Accuracy"), ("loss", "Loss")]
    fig, axes = plt.subplots(1, len(metrics), figsize=(14, 4))
    for ax, (metric, display_name) in zip(axes, metrics):
        for key, history in histories.items():
            epochs = range(1, len(history[metric]) + 1)
            ax.plot(epochs, history[metric], label=f"{key} train")
            ax.plot(epochs, history[f'val_{metric}'], linestyle='--', label=f"{key} val")
        ax.set_title(f"{title_prefix}{display_name}")
        ax.set_xlabel("Epoch")
        ax.set_ylabel(display_name)
    handles, labels = axes[-1].get_legend_handles_labels()
    axes[-1].legend(handles, labels, loc='upper right')
    plt.tight_layout()
    plt.show()

plot_history_curves(extreme_histories, title_prefix="Extreme models: ")


### What do we observe?

- **Underfit tiny network**: Training and validation accuracy stay low and the losses remain high, which tells us the model doesn't have enough capacity to represent the patterns in Fashion-MNIST.
- **Overfit huge network**: Training accuracy keeps climbing while validation accuracy plateaus and validation loss rises, showing the model memorizes the training subset but generalizes poorly.
- **Takeaway**: Effective architectures balance depth and width with regularization. Too few layers underfit, while an overpowered network without safeguards quickly overfits.


## Activation Function Experiments

Activations control how information flows between layers. We'll keep a two-layer architecture fixed and vary the nonlinearities to see how they impact convergence and final accuracy.

In [ ]:
activations = ["relu", "tanh", "sigmoid", "elu"]
activation_histories = {}
for act in activations:
    print(f"
Training activation variant: {act}")
    model = build_mlp([256, 128], activation=act)
    history = model.fit(
        train_x,
        train_y,
        validation_data=(val_x, val_y),
        epochs=5,
        batch_size=128,
        verbose=0
    )
    activation_histories[act] = history.history
    print(f"Final validation accuracy: {history.history['val_accuracy'][-1]:.3f}")

plt.figure(figsize=(7, 4))
for act, history in activation_histories.items():
    epochs = range(1, len(history["val_accuracy"]) + 1)
    plt.plot(epochs, history["val_accuracy"], marker="o", label=act)
plt.title("Validation accuracy by activation")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.tight_layout()
plt.show()

### Activation Takeaways

- `relu` (and variants like `elu`) often accelerate training in deeper networks because they mitigate vanishing gradients.
- Saturating activations (`sigmoid`, `tanh`) can struggle without careful initialization or normalization but may still perform competitively with fewer layers.
- Observe whether activations converge to similar validation accuracy despite different training trajectories.

> **Try it:** Mix activations across layers (e.g., first layer `relu`, second `tanh`) and see how the dynamics change.

## Guided Exercises

1. **Architecture tuning**: Modify `architectures` to include a very small network (e.g., `[64]`) and a very deep one (e.g., six layers). Compare validation accuracy and discuss bias/variance trade-offs.
2. **Regularization challenge**: Extend the new underfitting/overfitting section by adding dropout, `layers.BatchNormalization()`, or early stopping to the overfitting model. How much can you shrink the validation gap?
3. **Activation sweep**: Experiment with `swish` (`tf.nn.swish`) or `gelu`. You can pass a callable to the `activation` argument.
4. **TensorBoard logging (optional)**: Wrap training with `keras.callbacks.TensorBoard(log_dir="runs/week02")` to visualize losses and gradients. Record at least one screenshot of your TensorBoard scalars.
5. **Reflection**: For each experiment, note (a) the architecture, (b) activation(s), and (c) final training/validation accuracy. Summarize which design choices worked best and why.
